In [1]:
import pandas as pd
from covidb.pull import data_pull_csv
from covidb.process import get_todays_data

In [2]:
# print("Starting Pull")
# data_pull_csv()
# print("Finished Pull")
print("Started Processing")
data_dict = get_todays_data()
print("Finished Processing")

Started Processing
Finished Processing


In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
df = data_dict["all_data"]
df = df.loc[df["is_full_state"] == True]
df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Recovered,Deaths,is_full_state,county,state
0,Washington,US,47.4009,-121.4905,2020-01-22,0,0,0,True,NaN,Washington
1,Washington,US,47.4009,-121.4905,2020-01-23,0,0,0,True,NaN,Washington
2,Washington,US,47.4009,-121.4905,2020-01-24,0,0,0,True,NaN,Washington
3,Washington,US,47.4009,-121.4905,2020-01-25,0,0,0,True,NaN,Washington
4,Washington,US,47.4009,-121.4905,2020-01-26,0,0,0,True,NaN,Washington
...,...,...,...,...,...,...,...,...,...,...,...
13580,Virgin Islands,US,18.3358,-64.8963,2020-03-12,0,0,0,True,NaN,Virgin Islands
13581,Virgin Islands,US,18.3358,-64.8963,2020-03-13,0,0,0,True,NaN,Virgin Islands
13582,Virgin Islands,US,18.3358,-64.8963,2020-03-14,1,0,0,True,NaN,Virgin Islands
13583,Virgin Islands,US,18.3358,-64.8963,2020-03-15,1,0,0,True,NaN,Virgin Islands


In [30]:
melted = pd.melt(df, ["state", "Date", "Confirmed"])
melted

,state,Date,Confirmed,variable,value
0,Washington,2020-01-22,0,Province/State,Washington
1,Washington,2020-01-23,0,Province/State,Washington
2,Washington,2020-01-24,0,Province/State,Washington
3,Washington,2020-01-25,0,Province/State,Washington
4,Washington,2020-01-26,0,Province/State,Washington
...,...,...,...,...,...
23755,Virgin Islands,2020-03-12,0,county,NaN
23756,Virgin Islands,2020-03-13,0,county,NaN
23757,Virgin Islands,2020-03-14,1,county,NaN
23758,Virgin Islands,2020-03-15,1,county,NaN


In [15]:
sns.lineplot(x="Date", y="Confirmed", data=melted)
plt.show()

,state,Date,Confirmed,variable,value
0,Washington,2020-01-22,0,Province/State,Washington
1,Washington,2020-01-23,0,Province/State,Washington
2,Washington,2020-01-24,0,Province/State,Washington
3,Washington,2020-01-25,0,Province/State,Washington
4,Washington,2020-01-26,0,Province/State,Washington
...,...,...,...,...,...
108675,Virgin Islands,2020-03-12,0,county,NaN
108676,Virgin Islands,2020-03-13,0,county,NaN
108677,Virgin Islands,2020-03-14,1,county,NaN
108678,Virgin Islands,2020-03-15,1,county,NaN
